In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import gcsfs
import os
import matplotlib.pyplot as plt

#from keras.models import Sequential
#from keras.layers import Dense
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
%env PROJECT_ID=

In [ ]:
fs = gcsfs.GCSFileSystem(project=os.environ['PROJECT_ID'])
bucket_name = 'physionet_2009'
fs.ls(bucket_name)

In [ ]:
TRAIN_BATCH_SIZE = 32
NUM_TRAIN_EXAMPLES = 256871  # training dataset repeats, it'll wrap around
NUM_EVALS = 5  # how many times to evaluate
# Enough to get a reasonable sample, but not so much that it slows down
NUM_EVAL_EXAMPLES = 64218

CSV_COLUMNS = ['wave', 'start_window', 'end_window', 'avg_sys', 'avg_dias', 'avg_map',
       'current_hypotensive', 'hypotensive_in_15']
DEFAULTS = [[''],[0.0],[0.0],[0.0],[0.0],[0.0],
            [0.0],[0.0]]

LABEL_COLUMN = 'hypotensive_in_15'

In [ ]:
#training = get_training_labels()
training_data = pd.read_csv(fs.open(f'{bucket_name}/processed/processed_all.csv'))

training_data.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)
training_data.columns

In [ ]:
# Make dataset

In [ ]:
def features_and_labels(row_data):
    """Splits features and labels from feature dictionary.

    Args:
        row_data: Dictionary of CSV column names and tensor values.
    Returns:
        Dictionary of feature tensors and label tensor.
    """
    label = row_data.pop(LABEL_COLUMN)

    return row_data, label  # features, label


def load_dataset(pattern,csv_columns,defaults,label_column, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    """Loads dataset using the tf.data API from CSV files.

    Args:
        pattern: str, file pattern to glob into list of files.
        batch_size: int, the number of examples per batch.
        mode: tf.estimator.ModeKeys to determine if training or evaluating.
    Returns:
        `Dataset` object.
    """
    # Make a CSV dataset
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=csv_columns,
        column_defaults=defaults)
    print(type(dataset))

    # Map dataset to features and label
    dataset = dataset.map(map_func=features_and_labels)  # features, label
    
    # Shuffle and repeat for training
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # Take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(buffer_size=1)

    return dataset

In [ ]:
train_df = load_dataset(pattern = 'data/training_data_cleaned.csv',
                        csv_columns = CSV_COLUMNS,
                        defaults = DEFAULTS,
                        label_column = LABEL_COLUMN,
                        batch_size=TRAIN_BATCH_SIZE,
                        mode=tf.estimator.ModeKeys.TRAIN)



test_df = load_dataset(pattern = 'data/test_data_cleaned.csv',
                        csv_columns = CSV_COLUMNS,
                        defaults = DEFAULTS,
                        label_column = LABEL_COLUMN,
                        batch_size=TRAIN_BATCH_SIZE,
                        mode=tf.estimator.ModeKeys.EVAL)

In [ ]:
train_df

In [ ]:
# Create input layers

In [ ]:
def create_input_layers():
    """Creates dictionary of input layers for each feature.

    Returns:
        Dictionary of `tf.Keras.layers.Input` layers for each feature.
    """
    inputs = {
        colname: tf.keras.layers.Input(
            name=colname, shape=(), dtype="float32")
        for colname in ["avg_sys", "avg_dias", "current_hypotensive"]}
    '''
    inputs.update({
        colname: tf.keras.layers.Input(
            name=colname, shape=(), dtype="string")
        for colname in ["is_male", "plurality"]})
    '''

    return inputs

In [ ]:
# Create feature columns

In [ ]:
def categorical_fc(name, values):
    """Helper function to wrap categorical feature by indicator column.

    Args:
        name: str, name of feature.
        values: list, list of strings of categorical values.
    Returns:
        Indicator column of categorical feature.
    """
    cat_column = tf.feature_column.categorical_column_with_vocabulary_list(
            key=name, vocabulary_list=values)

    return tf.feature_column.indicator_column(categorical_column=cat_column)


def create_feature_columns():
    """Creates dictionary of feature columns from inputs.

    Returns:
        Dictionary of feature columns.
    """
    feature_columns = {
        colname : tf.feature_column.numeric_column(key=colname)
           for colname in ["avg_sys", "avg_dias", "current_hypotensive"]
    }
    '''
    feature_columns["is_male"] = categorical_fc(
        "is_male", ["True", "False", "Unknown"])
    feature_columns["plurality"] = categorical_fc(
        "plurality", ["Single(1)", "Twins(2)", "Triplets(3)",
                      "Quadruplets(4)", "Quintuplets(5)", "Multiple(2+)"])
    '''
    return feature_columns

In [ ]:
# Create hidden and output layer

In [ ]:
def get_model_outputs(inputs):
    """Creates model architecture and returns outputs.

    Args:
        inputs: Dense tensor used as inputs to model.
    Returns:
        Dense tensor output from the model.
    """
    # Create two hidden layers of [64, 32] just in like the BQML DNN
    h1 = tf.keras.layers.Dense(64, activation="relu", name="h1")(inputs)
    h2 = tf.keras.layers.Dense(32, activation="relu", name="h2")(h1)

    # Final output is a linear activation because this is regression
    output = tf.keras.layers.Dense(
        units=1, activation="sigmoid", name="weight")(h2)

    return output

In [ ]:
# Build model

In [ ]:
def build_dnn_model():
    """Builds simple DNN using Keras Functional API.

    Returns:
        `tf.keras.models.Model` object.
    """
    # Create input layer
    inputs = create_input_layers()

    # Create feature columns
    feature_columns = create_feature_columns()

    # The constructor for DenseFeatures takes a list of numeric columns
    # The Functional API in Keras requires: LayerConstructor()(inputs)
    dnn_inputs = tf.keras.layers.DenseFeatures(
        feature_columns=feature_columns.values())(inputs)

    # Get output of model given inputs
    output = get_model_outputs(dnn_inputs)

    # Build model and compile it all together
    model = tf.keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['AUC','Precision','Recall'])

    return model

print("Here is our DNN architecture so far:\n")
model = build_dnn_model()
print(model.summary())

In [ ]:
tf.keras.utils.plot_model(
    model=model, show_shapes=False, rankdir="LR")

In [ ]:

steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)


history = model.fit(
    train_df,
    validation_data=test_df,
    epochs=NUM_EVALS,
    steps_per_epoch=steps_per_epoch#,
    #callbacks=[tensorboard_callback]
)


In [ ]:
# Plot
import matplotlib.pyplot as plt
nrows = 1
ncols = 2
fig = plt.figure(figsize=(10, 5))

for idx, key in enumerate(["loss", "rmse"]):
    ax = fig.add_subplot(nrows, ncols, idx+1)
    plt.plot(history.history[key])
    plt.plot(history.history["val_{}".format(key)])
    plt.title("model {}".format(key))
    plt.ylabel(key)
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left");